# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geoviews as gv
import geoviews.feature as gf
import geoviews.tile_sources as gts
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,4.81,87,100,2.06,AR,1699673526
1,1,colonia,50.9333,6.9500,7.76,91,75,5.14,DE,1699673442
2,2,papatowai,-46.5619,169.4708,18.11,47,0,2.20,NZ,1699673407
3,3,puerto natales,-51.7236,-72.4875,4.35,95,100,2.46,CL,1699673487
4,4,mata-utu,-13.2816,-176.1745,28.56,77,98,3.95,WF,1699673660


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

api_key = geoapify_key

# Define a function to get the coordinates for a city using the Geoapify API
def get_coordinates(city):
    url = f"https://api.geoapify.com/v1/geocode/search?text={city}&apiKey={api_key}"
    response = requests.get(url).json()
    try:
        coordinates = response['features'][0]['geometry']['coordinates']
    except (KeyError, IndexError):
        # If the 'features' or 'geometry' key is missing, or the list of features is empty,
        # return a tuple of NaN values to indicate missing coordinates
        coordinates = (float('nan'), float('nan'))
    return coordinates


# Add a column to the DataFrame with the coordinates for each city
city_data_df["Coordinates"] = city_data_df["City"].apply(get_coordinates)

# Define a function to create a geoViews Element with a point for each city
def create_point(city_data):
    city, humidity, coordinates = city_data
    lon, lat = coordinates
    point = gv.Points([(lon, lat, humidity)]).opts(color="blue", size=humidity/10, tools=['hover'], hover_text=f"City: {city}, Humidity: {humidity}")
    return [point]

    # Create a geoViews Element with a point for each city
points = gv.Dataset(city_data_df[["City", "Humidity", "Coordinates"]].itertuples(index=False), kdims=["Longitude", "Latitude", "Humidity"]).to(gv.Points, ['Longitude', 'Latitude'], ['Humidity']).apply(create_point)
points = [*points]

# Create the map with the points and other elements
map = gv.tile_sources.CartoDark() * gf.coastline() * points

# Convert the Element to an Overlay object
overlay = map * gf.feature.ocean()


# Create a scatter plot using hvplot
scatter = city_data_df.hvplot.scatter(x='Lng', y='Lat', c='Humidity', size='Humidity', 
                            colormap='viridis', alpha=0.8, hover_cols=['City'])

# Display the plot using the show() method
scatter.show()

NotImplementedError: Iteration on Elements is not supported.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria
ideal_cities = city_data_df[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
ideal_cities = ideal_cities.dropna()

# Display sample data
ideal_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Coordinates
45,45,tura,25.5198,90.2201,25.19,61,0,0.23,IN,1699673506,"[19.5949442, 47.60935]"
65,65,yanbu,24.0891,38.0637,26.02,73,0,3.09,SA,1699673682,"[56.5359679, 24.2545689]"
105,105,supaul,26.1167,86.6000,25.16,48,0,1.86,IN,1699673696,"[86.6008021, 26.1186598]"
121,121,coremas,-7.0144,-37.9458,25.53,57,0,3.78,BR,1699673701,"[-37.9465062, -7.0175897]"
158,158,morondava,-20.2833,44.2833,24.41,64,0,2.19,MG,1699673717,"[44.2786121, -20.2951102]"


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ushuaia,AR,-54.8000,-68.3000,87,
1,colonia,DE,50.9333,6.9500,91,
2,papatowai,NZ,-46.5619,169.4708,47,
3,puerto natales,CL,-51.7236,-72.4875,95,
4,mata-utu,WF,-13.2816,-176.1745,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
# Set parameters to search for a hotel
radius = 10000
params = {
    "type": "hotel",
    "radius": radius,
    "limit": 1,
    "sort": "distance",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lon},{lat}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
 # Convert the API response to JSON format
    name_address = json.dumps(name_address, indent=4, sort_keys=True)
    name_address = json.loads(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df




Starting hotel search
ushuaia - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
mata-utu - nearest hotel: No hotel found
touros - nearest hotel: No hotel found
mount gambier - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
wanning - nearest hotel: No hotel found
puerto ayora - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
namsos - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
west odessa - nearest hotel: No hotel found
zaozersk - nearest hotel: No hotel found
mossel bay - nearest hotel: No hotel found
potosi - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
bargarh - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
hawai

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ushuaia,AR,-54.8000,-68.3000,87,No hotel found
1,colonia,DE,50.9333,6.9500,91,No hotel found
2,papatowai,NZ,-46.5619,169.4708,47,No hotel found
3,puerto natales,CL,-51.7236,-72.4875,95,No hotel found
4,mata-utu,WF,-13.2816,-176.1745,77,No hotel found
...,...,...,...,...,...,...
564,muisne,EC,0.6000,-80.0333,83,No hotel found
565,shenjiamen,CN,29.9576,122.2980,70,No hotel found
566,qui nhon,VN,13.7667,109.2333,78,No hotel found
567,abqaiq,SA,25.9371,49.6776,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)